In [1]:
import pandas as pd
import pandas as pd
import numpy as np
from transformers import AutoModelForSequenceClassification
from transformers import TextClassificationPipeline
from transformers import AutoTokenizer  # Or BertTokenizer
import torch

d:\Programas\anaconda3\envs\datasets_iat\Lib\site-packages\tqdm\auto.py:21: TqdmWarning: IProgress not found. Please update jupyter and ipywidgets. See https://ipywidgets.readthedocs.io/en/stable/user_install.html
  from .autonotebook import tqdm as notebook_tqdm


In [4]:
judgements_1 = pd.read_csv(r'D:\Proyectos\amag\classification\non_appealed_judgements_2018.csv')

In [5]:
import numpy as np

# Assuming df is your dataframe and 'column_name' is the column you want to check
judgements_1['judge_from_opinion'] = judgements_1['judge_from_opinion'].apply(lambda x: np.nan if isinstance(x, str) and len(x) > 40 else x)

In [ ]:
def remove_special_chars(text):
    if isinstance(text, str):
        cleaned_text = ''
        for char in text:
            if char in 'ÁÉÍÓÚ':
                # Replace uppercase vowels with their corresponding lowercase counterparts
                if char == 'Á':
                    cleaned_text += 'A'
                elif char == 'É':
                    cleaned_text += 'E'
                elif char == 'Í':
                    cleaned_text += 'I'
                elif char == 'Ó':
                    cleaned_text += 'O'
                elif char == 'Ú':
                    cleaned_text += 'U'
            else:
                cleaned_text += char
        return cleaned_text
    else:
        return text  # Return non-string values as is

# Apply function to DataFrame column
judgements_1['judge_from_opinion'] = judgements_1['judge_from_opinion'].apply(remove_special_chars)

In [48]:
hombres = pd.read_csv(r"D:\Proyectos\amag\pruebas\hombres.csv")
mujeres = pd.read_csv(r"D:\Proyectos\amag\pruebas\mujeres.csv")
apellidos = pd.read_csv(r"D:\Proyectos\amag\pruebas\apellidos.csv")

In [58]:
nombres = pd.concat([hombres, mujeres, apellidos], axis=0, ignore_index=True)
nombres = nombres.drop(columns=['frec', 'edad_media','apellido','frec_pri','frec_seg','freq_rep'])
nombres = nombres.dropna()

In [59]:
names_to_add = ['FRESIA', 'DEYVI', 'CUEVA', 'RAUL', 'CUENTAS', 'GUEVARA', 
                'HURTADO', 'CORDOVA', 'PADILLA', 'GAVIDIA', 'ESPINOZA', 'DIAZ', 
                'JULCA', 'LEON', 'AQUINO', 'LIZ', 'GUERRA', 'MERCADO', 'AGUILAR',
                'HERRERA', 'CALLE', 'CORONADO', 'CORNEJO','RAMIREZ']


In [60]:
nombres_list= nombres['nombre'].tolist()

In [61]:
nombres_list = nombres_list + names_to_add

In [62]:
nombres_list[-30:]

['YOVANKA',
 'YUFEI',
 'YUQI',
 'YURI MARCELA',
 'YURONG',
 'YUSRAA',
 'YUXIANG',
 'YUYUN',
 'ZAMBRA',
 'ZARAH',
 'ZOE MARIE',
 'ZOHORA',
 'ZOI',
 'ZOILA ROCIO',
 'FRESIA',
 'DEYVI',
 'CUEVA',
 'RAUL',
 'CUENTAS',
 'GUEVARA',
 'HURTADO',
 'CORDOVA',
 'PADILLA',
 'GAVIDIA',
 'ESPINOZA',
 'DIAZ',
 'JULCA',
 'LEON',
 'AQUINO',
 'LIZ']

In [18]:
df_new = judgements_1[['case_file','judge','judge_from_opinion']]
df_new

,case_file,judge,judge_from_opinion
0,00006-2018-0-0210-JP-FC-01,ALVAREZ ACERO ROCIO DELSY,NaN
1,00006-2018-0-0215-JP-FC-01,HINOSTROZA HUERTA HILDA MARIA,NaN
2,00007-2018-0-0201-JP-FC-02,VILLAFAN CANO HOMMER FREY,NaN
3,00008-2018-0-0201-JP-FC-01,"HUERTA BOJORQUEZ, CARMEN EUGENIA",NaN
4,00022-2018-0-0201-JP-FC-01,RETUERTO TUEROS BESSI YOHANA,NaN
...,...,...,...
31687,00030-2018-0-3301-SP-CI-01,NaN,NaN
31688,00031-2018-0-3301-SP-CI-01,NaN,NaN
31689,00035-2018-0-3301-SP-CI-01,NaN,NaN
31690,00005-2018-0-3301-SP-FC-01,NaN,NaN


In [15]:
tokenizer = AutoTokenizer.from_pretrained('dccuchile/bert-base-spanish-wwm-uncased')
bert_model = AutoModelForSequenceClassification.from_pretrained("JFernandoGRE/classificator_names")
pipe_model = TextClassificationPipeline(model=bert_model, tokenizer=tokenizer)

# Define a function to get the score for a given text
def get_score(text):
    if pd.isna(text):  # Check if text is NaN
        return np.nan  # Or return a default score, e.g., 0 or -1
    return pipe_model(text, return_all_scores=True)[0][1]['score']

# Apply the function to the 'JUDGE_FROM_OPINION' column and create the 'SCORE' column
df_nan_judge['score'] = df_nan_judge['judge_from_opinion'].apply(get_score)


d:\Programas\anaconda3\envs\datasets_iat\Lib\site-packages\transformers\pipelines\text_classification.py:104: UserWarning: `return_all_scores` is now deprecated,  if want a similar functionality use `top_k=None` instead of `return_all_scores=True` or `top_k=1` instead of `return_all_scores=False`.
  warnings.warn(
C:\Users\PC\AppData\Local\Temp\ipykernel_19164\1386623876.py:10: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame.
Try using .loc[row_indexer,col_indexer] = value instead

See the caveats in the documentation: https://pandas.pydata.org/pandas-docs/stable/user_guide/indexing.html#returning-a-view-versus-a-copy
  df_nan_judge['score'] = df_nan_judge['judge_from_opinion'].apply(get_score)


In [10]:
df_nan_judge

,case_file,judge,judge_from_opinion,score
10,00040-2018-0-0201-JP-FC-01,"HUERTA BOJORQUEZ, CARMEN EUGENIA","AGUILAR DEXTRE, HERNANDO EDGAR",0.999557
13,00050-2018-0-0202-JP-FC-01,ROBLES TORRES HIBETH LEGNNY,ROBLES LAZARO MONER FELIX,0.999583
15,00055-2018-0-0212-JP-FC-01,MENDOZA ASENCIOS WILBER,WILBER MENDOZA ASENCIOS,0.999571
16,00058-2018-0-0201-JP-FC-01,"HUERTA BOJORQUEZ, CARMEN EUGENIA","AGUILAR DEXTRE, HERNANDO EDGAR",0.999557
20,00070-2018-0-0201-JP-FC-02,SANTISTEBAN VALENZUELA AILLENY ADELA,MAGDALENA SALAZAR SOTO,0.999556
...,...,...,...,...
31618,00549-2018-0-3301-JP-FC-01,CAJAS PEREZ JUAN DIEGO,ATENCIO VALLADARES KARINA MARICRUZ,0.999594
31627,00575-2018-0-3301-JP-FC-01,"PAREDES VILLA, LIDIA VICTORIA",ATENCIO VALLADARES KARINA MARICRUZ,0.999594
31638,00613-2018-0-3301-JP-FC-01,ANGELA RENGIFO CARPIO,ANGELA RENGIFO CARPIO,0.999581
31681,00164-2018-0-3301-JR-CI-01,ESTELA SOLANO ALEJOS,ROY ESTEBAN ALVA NAVARRO,0.999607


In [16]:
df_nan_judge.loc[df_nan_judge['score'] < 0.998, 'judge_from_opinion'] = np.nan  # Set NaN where score > 0.998

# Copy content from 'judge' to 'judge_from_opinion' where 'judge_from_opinion' is empty
df_nan_judge['judge_from_opinion'].replace('', np.nan, inplace=True)  # Ensure empty strings are treated as NaN
df_nan_judge['judge_from_opinion'].fillna(df_nan_judge['judge'], inplace=True)

C:\Users\PC\AppData\Local\Temp\ipykernel_19164\282632229.py:4: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame

See the caveats in the documentation: https://pandas.pydata.org/pandas-docs/stable/user_guide/indexing.html#returning-a-view-versus-a-copy
  df_nan_judge['judge_from_opinion'].replace('', np.nan, inplace=True)  # Ensure empty strings are treated as NaN
C:\Users\PC\AppData\Local\Temp\ipykernel_19164\282632229.py:5: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame

See the caveats in the documentation: https://pandas.pydata.org/pandas-docs/stable/user_guide/indexing.html#returning-a-view-versus-a-copy
  df_nan_judge['judge_from_opinion'].fillna(df_nan_judge['judge'], inplace=True)


In [17]:
df_nan_judge.to_excel(r'D:\Proyectos\amag\pruebas\judges_names.xlsx', index=False)

In [58]:
cell_value = df_new.at[14886, 'opinion_text']
cell_value

'         CORTE SUPERIOR DE JUSTICIA DE LAMBAYEQUE\n         \nCORTE SUPERIOR DE JUSTICIA DE LAMBAYEQUE    \n         \n         \nCORTE SUPERIOR DE JUSTICIA DE LAMBAYEQUE\nCORTE SUPERIOR DE JUSTICIA DE LAMBAYEQUE\n\nPRIMER JUZGADO DE PAZ LETRADO CIVIL ----    CHICLAYO\n          PRIMER JUZGADO DE PAZ LETRADO CIVIL \n          \nCHICLAYO    \nPRIMER JUZGADO DE PAZ LETRADO CIVIL \n          \n          \nPRIMER JUZGADO DE PAZ LETRADO CIVIL \nCHICLAYO\nCHICLAYO\n\n: 00039\n2018----0000----1706\n: 00039----2018\n1706----JPJPJPJP----CICICICI----01010101    \n17061706\n20182018\n: 00039\n: 00039\n: PEDRO HIPOLITO DEJO VALLEJOS\n: PEDRO HIPOLITO DEJO VALLEJOS    \n: PEDRO HIPOLITO DEJO VALLEJOS\n: PEDRO HIPOLITO DEJO VALLEJOS\n\n    \nEXPEDIENTE\nEXPEDIENTE        \nEXPEDIENTE\nEXPEDIENTE\nDEMANDANTE\nDEMANDANTE    \nDEMANDANTE\nDEMANDANTE\n:EMPRESA DE SERVICIOS FUNERARIOS RAMIREZ E.I.R.LR.LR.LR.L    \n            :EMPRESA DE SERVICIOS FUNERARIOS RAMIREZ E.I.\nDEMANDADO \nDEMANDADO         \

In [9]:
df_nan_judge = df_nan_judge[df_nan_judge['judge_from_opinion'].isna()]
df_nan_judge

,judge,opinion_text,judge_from_opinion
236,NaN,1° SALA CIVIL - Sede Central \n\nEXPEDIENTE \n...,NaN
794,NaN,JUZGADO DE PAZ LETRADO DE ORCOPAMPA \nEXPEDIEN...,NaN
815,NaN,C O R T E S U P E R I O R D E J ...,NaN
880,NaN,NaN,NaN
887,NaN,EXPEDIENTE \nDEMANDANTE \nDEMANDADO \nMATERIA ...,NaN
...,...,...,...
31687,NaN,1 \n\n \n\nC O R T E S U P E R I O R D E ...,NaN
31688,NaN,\n \n\n \n \nEXPEDIENTE \n \nRELATOR \nMATERI...,NaN
31689,NaN,\n \n\n \n\n \n \nEXPEDIENTE \n \nMATERIA \n ...,NaN
31690,NaN,CORTE SUPERIOR DE JUSTICIA DE VENTANILLA Y \n\...,NaN


In [10]:
cell_value = df_nan_judge.at[236, 'opinion_text']
cell_value

'1° SALA CIVIL - Sede Central \n\nEXPEDIENTE \n\n: 00002-2018-0-0201-SP-CI-01 \n\nMATERIA  \n\nRELATOR \n\n \n\n \n\n: ACCION CONTENCIOSA ADMINISTRATIVA \n\n: ASIS SAENZ LEONCIO GABRIEL \n\nDEMANDADO \n\n: SOCIEDAD DE BENEFICIENCIA PUBLICA DE \n\nCHIQUIAN \n\nDEMANDANTE  \n\n: PEREZ DURAN, VILA \n\n \n\nRESOLUCION N. º11 \nHuaraz, veintitrés de marzo del año dos mil dieciocho \n \n\nVISTOS  en  audiencia  pública,  con  la  certificación  que  obra  a  fojas  doscientos \n\ncuarentaisiete,  remitido  por  el  Juzgado  Mixto  de  Bolognesi,  a  la  vista  del \n\ndictamen  emitido  por  el  señor  Fiscal  Superior  de  la  Fiscalía  Superior  Civil  y \n\nFamilia del Ministerio Público en el dictamen de fojas doscientos treintaicuatro \n\na doscientos cuarentaicuatro, para resolver. \n\nASUNTO:  \n\nRecurso  de  apelación  interpuesto  por  Vila  Pérez  Duran  contra  la  sentencia \n\ncontenida en la resolución número ocho de fecha dos de octubre del año dos mil \n\ndiecisiete,  que  f